Urban Data Science & Smart Cities <br>
URSP688Y Spring 2025<br>
Author: Huan Zhou <br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

# Assessing streetscape quality related to pedestrian crashes using self-supervised learning

#### 1. Question
The study aims to develop a self-supervised learning model to assess streetscape quality related to pedestrian crashes in Baltimore City. The project wants to assess the streetscape quality at a large scale based on street view images.

#### 2. Methods
The methods include four key steps: street view images collection, labeling, self-supervised learning training, and evaluation.

##### Pseudocode

    Step 1. Download street view images from Google Maps Platform using Street View Static API, captured in four different directions (90°, 180°, 270°, and 360°)

    Step 2. Label images manually based on strict rubric, and employ ChatGPT to generate pseudo-labels. 

    Step 3. Employ a convolutional neural network (CNN) like the ResNet-50 to train the model.

    Step 4. Evaluate the model’s performance based on accuracy, precision, recall, and F1-score.


#### 3. Data
The data includes 1200 Baltimore City’s street view images, which are download from Google Maps Platform using API. 

The dataset is split as the train, validation, and test dataset. The train and validation dataset may total 1,100 Baltimore street view images (generated by ChatGPT, persudo-labeled images), with a ratio of 9:1 for training and validation. The test data contains 100 street view images (manually annotated), which have ground truth.

## Step 1. Download Street view images
First, we create sampling points every 100 meters of road networks. Then, <br>
Step 1.1. Request panoid, fetch all available panoramas near the samling points.<br>
Step 1.2. download street view iamges, using panoid with the latest date.<br>

### Step 1.1. fetch all available panoramas near the samling points

In [11]:
!pip install streetview

In [12]:
! pip install tqdm

In [13]:
import pandas as pd
from streetview import search_panoramas
from tqdm import tqdm

In [9]:
# Input & Output Files
input_file = "100point_network_test.csv"
output_file = "100points_panoid_results.csv"

df = pd.read_csv(input_file)

# Define CSV Columns (Including all attributes)
columns = ["id", "Latitude", "Longitude", "Pano_ID", "Pano_Lat", "Pano_Lon",
           "Heading", "Pitch", "Roll", "Date", "Elevation"]

# Initialize empty file with headers
pd.DataFrame(columns=columns).to_csv(output_file, index=False)

#  Process Data
results = []  # Store results in a list before saving

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Panoramas"):
    try:
        # Fetch all available panoramas for the location
        panos = search_panoramas(lat=row['latitude'], lon=row['longitude'])

        if panos:
            for pano in panos:
                results.append([
                    row['id'], row['latitude'], row['longitude'],  # Original input data
                    pano.pano_id, pano.lat, pano.lon,             # Panorama location
                    pano.heading, pano.pitch, pano.roll,          # Camera orientation
                    pano.date, pano.elevation                     # Image metadata
                ])
        else:
            # No panorama found → Store as None
            results.append([row['id'], row['latitude'], row['longitude'], None, None, None, None, None, None, None, None])

    except Exception as e:
        # Error handling
        results.append([row['id'], row['latitude'], row['longitude'], "Error", None, None, None, None, None,None, str(e)])

    #  Save progress after every location
    pd.DataFrame(results, columns=columns).to_csv(output_file, mode='a', header=False, index=False)
    results = []  # Clear buffer after writing to file

Processing Panoramas: 100%|███████████████████| 132/132 [00:39<00:00,  3.37it/s]


### Step 1.2. Download street view
Using 'panoid_unique_latest_date.py',we remove duplicate panoids and obtain the panoids of the latest date, and we get the "panoid_unique_latest_date.csv". 

Based on these panoids, we download street view images using "download_gsv.py". Here is the key parameters.

##### Key parameters of requesting Google Street View images
1) Width and height: the dimensions of the output image in pixels. For non-premium downloads, both the width and height have a maximum limit of 640 pixels. 

2) Heading: the compass direction that the camera is facing, where 0° is north, 90° is east, 180° is south, and 270° is west.

3) FOV (field of view): the extent of the scene captured in the image; a smaller FOV, such as 30 degrees, results in a zoomed-in image where objects appear larger and occupy more of the frame, while a larger FOV provides a wider, more expansive view.

4) Pitch: controls the vertical angle of the camera relative to the horizon — a pitch of 0° means the camera is level with the ground, positive values tilt the camera upward, and negative values tilt it downward.

Because the API key is missing and images are too large, we cannot run the code and show the results of "download_gsv.py". Part of images are shown in the folder "street_view_image_samples".

## Step 2. Label Street view images
First, we create rubrics of labeling images based on literature review and make assumptions related to pedestrian crashes. Then, we label images manually based on strict the rubrics, and employ gpt-4.1 model to generate pseudo-labels. 

## Step 2.1. Create rubrics of labeling images

The project identifies seven key indicators that are hypothesized to be closely associated with pedestrian crashes based on literature reviews.

#### Table 1. Hypotheses of street view indicators related to pedestrian crashes
| Indicators                  | Hypothesis |
|:----------------------------|:-----------|
| Road Pavement Condition     | Poor road pavement conditions reduces driver maneuverability, increase pedestrian crash risks. |
| Presence of Sidewalk        | The absence of sidewalks forces pedestrians to share space with vehicles, increasing their exposure to moving traffic and elevating crash risk. |
| Sidewalk Condition          | Poor sidewalk conditions (e.g., uneven, broken surfaces) increase tripping risk, discourage sidewalk use and push pedestrians onto roadways, leading to higher chances of vehicle-pedestrian conflicts. |
| Sidewalk Buffer Zone        | The presence of a sidewalk buffer zone (e.g., trees, landscaping, bike lanes) physically separates pedestrians from vehicles, reducing the likelihood of crashes. |
| Crosswalk Presence          | Clearly marked crosswalks provide predictable crossing locations for pedestrians, improving driver awareness and reducing the probability of midblock crossing accidents. |
| Traffic Signs Presence      | Visible traffic signs (e.g., pedestrian crossing warnings) alert drivers to pedestrian activity, enhancing compliance with traffic rules and reducing crash risks. |
| Streetlight Presence        | Adequate streetlighting improves visibility for both pedestrians and drivers at night, reducing crash likelihood in low-light conditions. |




#### Table 1. Rubric of labeling streetscape quality

| Category               | Indicators                    | Labels            | Description                                   | Criteria |
|-------------------------|-----------------------------|-------------------|----------------------------------------------|----------|
| Road                    | **Road pavement condition** | Poor, Fair, Good  | Quality of the road surface                  | **Poor**: Large potholes, deep cracks, uneven surfaces, under construction, large patches. Driving conditions are very uncomfortable, bumpy, potentially impassable, and unsafe.<br>**Fair**: Several areas of distress, with more than one form of cracking, rutting, shoving, etc. Driving conditions are somewhat uncomfortable, with vibrations and slight leaning in curvature, etc.<br>**Good**: Small areas of cracks, potholes, bleeding, etc., minor surface vibration, but driving conditions are generally smooth and comfortable. |
| Pedestrian infrastructure | **Presence of sidewalk**     | Yes, No           | Presence of sidewalk                         |          |
|                         | **Sidewalk condition**      | Poor, Fair, Good  | Quality of pedestrian sidewalks              | **Poor**: Large crack, uneven surfaces, missing sections, too narrow, or under construction, hazardous conditions for pedestrians, potentially leading to trips and falls. Surface is very uncomfortable to walk on, with significant obstructions or debris.<br>**Fair**: Several areas of distress, such as cracking, unevenness, or minor obstructions, but surface is passable, and walking conditions are somewhat uncomfortable.<br>**Good**: Minor cracks, smooth surface, well-maintained, and surface is comfortable for walking, with minimal obstructions. |
|                         | **Sidewalk buffer zone**     | Yes, No           | Physical barriers between the sidewalk and roadway (such as street trees, landscaping, bike lanes and parked cars) |          |
|                         | **Crosswalk presence**      | Yes, No           | Presence of pedestrian crossings             |          |
| Street furniture        | **Traffic signs presence**  | Yes, No           | Visibility of traffic signs                  |          |
|                         | **Streetlight presence**    | Yes, No           | Visibility of streetlights                   |          |


## Step 2.2. Label images manually and employ ChatGPT
The study labels 100 images manually based on the rubric strictly. Then we employ gpt-4.1 model to label about 1100 images. The code is shown in "chatgpt_label.py".

## Step 3. Train the multi-label image classification model
The project is using Vertex AI to train an multi-label image classification model.

## Step 3.1. Prepare train, validation, and test dataset
The training and validation datasets contain 1,100 street view images in a 9:1 ratio, labeled by the GPT-4.1 model. The labels are shown in "train_labels_chatgpt41.csv"

The test dataset contains 100 manually labeled street view images. The labels are shown in "test_labels_ground_truth.csv".


In [14]:
df_train = pd.read_csv("train_labels_chatgpt41.csv")
df_train.head()

,gs://baltimore_train_gsv/Ryutfzz40JBwaKhj1s9_og_fov90_180.jpg,road_pavement_condition_fair,presence_of_sidewalk_yes,sidewalk_condition_fair,presence_of_sidewalk_buffer_zone_no,crosswalk_presence_no,traffic_signs_yes,streetlight_presence_no
0,gs://baltimore_train_gsv/tZ50HlO1Noc3S1H3-mrHx...,road_pavement_condition_fair,presence_of_sidewalk_yes,sidewalk_condition_good,presence_of_sidewalk_buffer_zone_no,crosswalk_presence_no,traffic_signs_yes,streetlight_presence_no
1,gs://baltimore_train_gsv/IpYOmWo8C4SFhZSdnLPHE...,road_pavement_condition_fair,presence_of_sidewalk_yes,sidewalk_condition_good,presence_of_sidewalk_buffer_zone_no,crosswalk_presence_no,traffic_signs_no,streetlight_presence_no
2,gs://baltimore_train_gsv/SYNT22d_3Tm7drcINDgHl...,road_pavement_condition_fair,presence_of_sidewalk_yes,sidewalk_condition_good,presence_of_sidewalk_buffer_zone_no,crosswalk_presence_yes,traffic_signs_yes,streetlight_presence_yes
3,gs://baltimore_train_gsv/VBGYabkgv-pRTo08l5NF_...,road_pavement_condition_fair,presence_of_sidewalk_yes,sidewalk_condition_fair,presence_of_sidewalk_buffer_zone_yes,crosswalk_presence_no,traffic_signs_yes,streetlight_presence_yes
4,gs://baltimore_train_gsv/4V1SiVvliy-qHgXeo7jzH...,road_pavement_condition_fair,presence_of_sidewalk_yes,sidewalk_condition_fair,presence_of_sidewalk_buffer_zone_no,crosswalk_presence_no,traffic_signs_no,streetlight_presence_no


In [15]:
df_train = pd.read_csv("test_labels_ground_truth.csv")
df_train.head()

,gs://baltimore_real_test_gsv/_27ENXKT_T-PDeC893a6iQ_0.jpg,road_pavement_condition_fair,presence_of_sidewalk_yes,sidewalk_condition_fair,presence_of_sidewalk_buffer_zone_yes,crosswalk_presence_no,traffic_signs_no,streetlight_presence_yes
0,gs://baltimore_real_test_gsv/_27ENXKT_T-PDeC89...,road_pavement_condition_none_of_the_above,presence_of_sidewalk_yes,sidewalk_condition_good,presence_of_sidewalk_buffer_zone_yes,crosswalk_presence_no,traffic_signs_no,streetlight_presence_no
1,gs://baltimore_real_test_gsv/_27ENXKT_T-PDeC89...,road_pavement_condition_fair,presence_of_sidewalk_yes,sidewalk_condition_fair,presence_of_sidewalk_buffer_zone_yes,crosswalk_presence_no,traffic_signs_yes,streetlight_presence_yes
2,gs://baltimore_real_test_gsv/_27ENXKT_T-PDeC89...,road_pavement_condition_none_of_the_above,presence_of_sidewalk_yes,sidewalk_condition_good,presence_of_sidewalk_buffer_zone_yes,crosswalk_presence_no,traffic_signs_no,streetlight_presence_no
3,gs://baltimore_real_test_gsv/0HNOMYCm298-8mNHT...,road_pavement_condition_fair,presence_of_sidewalk_yes,sidewalk_condition_fair,presence_of_sidewalk_buffer_zone_yes,crosswalk_presence_yes,traffic_signs_no,streetlight_presence_no
4,gs://baltimore_real_test_gsv/0HNOMYCm298-8mNHT...,road_pavement_condition_poor,presence_of_sidewalk_yes,sidewalk_condition_poor,presence_of_sidewalk_buffer_zone_no,crosswalk_presence_no,traffic_signs_no,streetlight_presence_no


## Step 4. Evaluate the model
Evaluate the image classification model